In [2]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob 
from PIL import Image
import cv2
from skimage.filters import threshold_otsu
import torch
import pyperlin
import os

/Users/isaac/miniforge3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
n = 1024
shape = (n, 512, 512)
l = 2
p = 0.4

resolutions = [(l**i, l**i) for i in range(1,7)]
factors = [p**i for i in range(6)]

generator = torch.Generator()
generator.manual_seed(0)

fp = pyperlin.FractalPerlin2D(shape, resolutions, factors, generator=generator)
noise = fp()
noise = noise.numpy()
noise = (noise > threshold_otsu(noise)).astype(int)
for i,generation in enumerate(noise):
    plt.imsave(f"noise/gt/{i:04d}.png", generation, cmap="gray")
    plt.imsave(f"texture/gt/{i:04d}.png", generation, cmap="gray")
gts = noise

In [5]:
# Noise test images
noise_levels = [0.5, 1, 2, 4, 8]
noise_levels = [4, 8]
for im_num, gt in enumerate(gts):
    for noise_level in noise_levels:
        noise = np.random.normal(0, noise_level, gt.shape)
        noisy = gt + noise
        noisy -= noisy.min()
        noisy /= noisy.max()
        noisy *= 255
        noisy = Image.fromarray(noisy).convert("L")
        noisy.save(f'noise/{noise_level:.1f}/{im_num:04d}.png')

In [6]:
# Texture test images
def preprocess_texture(texture):
    desired_crop = (10, 10, 10, 10)
    # center crop image
    texture = texture[desired_crop[0]:-desired_crop[1], desired_crop[2]:-desired_crop[3]]
    im = Image.fromarray(texture).convert("L")
    im = im.resize((512, 512), resample=Image.BILINEAR)
    return np.array(im)

texture_pairs = [("1.1.12","1.1.13"),("1.2.02","1.2.04"),("1.2.07","1.2.09")]
for im_num, gt in enumerate(gts):
    for pair_num, texture_pair in enumerate(texture_pairs):
        texture1 = plt.imread(f"../textures/{texture_pair[0]}.tiff")
        texture2 = plt.imread(f"../textures/{texture_pair[1]}.tiff")
        texture1 = preprocess_texture(texture1)
        texture2 = preprocess_texture(texture2)
        mask = cv2.GaussianBlur(gt.astype(float), (5, 5), 0)
        blended = (texture1 * mask) + (texture2 * (1-mask))
        blended = Image.fromarray(blended).convert("L")
        blended.save(f'texture/images/{im_num}_{pair_num}.png')

In [32]:
np.random.seed(0)
cloud_ids = [os.path.basename(path).split(".")[0] for path in glob("clouds/images/*.png")]
k = 256

mini_test_ids = ["0003", "0029", "0053"]
test_ids = mini_test_ids + list(np.random.choice(cloud_ids, k + 3, replace=False))
test_ids = list(dict.fromkeys(test_ids))[:k]

train_ids = [id for id in cloud_ids if id not in test_ids]

# write train and test ids to file
with open("clouds/train_ids.txt", "w") as f:
    f.write("\n".join(train_ids))
with open("clouds/minitest_ids.txt", "w") as f:
    f.write("\n".join(mini_test_ids))
with open("clouds/test_ids.txt", "w") as f:
    f.write("\n".join(test_ids))
